In [2]:
import os
import json
from glob import glob

def read_jsonl(path):
    ret = []
    with open(path) as f:
        for line in f:
            ret.append(json.loads(line))
    return ret

os.makedirs("sampled_data_rebalanced", exist_ok=True)

for path in glob("sampled_data/*.jsonl"):
    questions = read_jsonl(path)
    for i, q in enumerate(questions):
        if i % 2 == 0 and q["answerKey"] == "A":
            q["question"]["choices"] = q["question"]["choices"][::-1]
            q["answerKey"] = "B"
        if i % 2 == 1 and q["answerKey"] == "B":
            q["question"]["choices"] = q["question"]["choices"][::-1]
            q["answerKey"] = "A"
        
        q["question"]["choices"][0]["label"] = "A"
        q["question"]["choices"][1]["label"] = "B"

    new_path = path.replace("sampled_data", "sampled_data_rebalanced")
    with open(new_path, "w") as f:
        for q in questions:
            f.write(json.dumps(q) + "\n")

In [4]:
!cat sampled_data_rebalanced/*.jsonl > sampled_data_rebalanced.jsonl
!wc -l sampled_data_rebalanced.jsonl

     500 sampled_data_rebalanced.jsonl


In [8]:
"""
,prompt,classes,answer_index
0,"Question: {Q}
A. {A1}
B. {A2}
Answer:","[' A', ' B']",0/1
1,"Question: {Q}
A. {A1}
B. {A2}
Answer:","[' A', ' B']",0/1
"""
import json, csv

with open("sampled_data_rebalanced.jsonl") as fin, open("sampled_data_rebalanced.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(["", "prompt", "classes", "answer_index"])
    for i, line in enumerate(fin):
        q = json.loads(line)
        Q = q["question"]["stem"]
        A1 = q["question"]["choices"][0]["text"]
        A2 = q["question"]["choices"][1]["text"]
        assert "\"" not in Q
        prompt = f"""The following are multiple choice questions (with answers) about common sense.
        
Question: {Q}
A. {A1}
B. {A2}
Answer:"""
        classes = "[' A', ' B']"
        answer_index = (0 if q["answerKey"] == "A" else 1)
        csvwriter.writerow([i, prompt, classes, answer_index])

In [13]:
"""
,prompt,classes,answer_index
0,"Question: {Q}
A. {A1}
B. {A2}
Answer:","[' A', ' B']",0/1
1,"Question: {Q}
A. {A1}
B. {A2}
Answer:","[' A', ' B']",0/1
"""
import json, csv

with open("sample_not_be_300.jsonl") as fin, open("sample_not_be_300_short.csv", "w") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    csvwriter.writerow(["", "prompt", "classes", "answer_index"])
    for i, line in enumerate(fin):
        q = json.loads(line)
        Q = q["question"]["stem"]
        A1 = q["question"]["choices"][0]["text"]
        A2 = q["question"]["choices"][1]["text"]
        assert "\"" not in Q and "\"" not in A1 and "\"" not in A2
        prompt = f"""The following are multiple choice questions about common sense.

Question: {Q}
Answer:"""
        classes = f"[\" {A1}\", \" {A2}\"]"
        answer_index = (0 if q["answerKey"] == "A" else 1)
        csvwriter.writerow([i, prompt, classes, answer_index])